In [12]:
!pip install langchain_openai langchain_core langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00


In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [6]:
response = llm.invoke("Qual a previsão do tempo em Recife-PE?")
print(response.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a previsão do tempo. Recomendo que você consulte um site de meteorologia ou um aplicativo de clima para obter as informações mais atualizadas sobre o tempo em Recife-PE.


In [7]:
a = 18273840567
b = 12830192835

response = llm.invoke("Quanto é 18273840567*12830192835? Responda apenas a o resultado.")
print(response.content)
print()
print(f'Resposta esperada:', a*b)

234084217173156999395.

Resposta esperada: 234456898310655737445


In [8]:
import random
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Soma dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois inteiros.

    Args:
        a: Primeiro inteiro
        b: Segundo inteiro
    """
    return a * b

@tool
def get_weather(city: str) -> str:
    """Retorna um clima aleatório para uma cidade.

    Args:
        city: Nome da cidade
    """
    weather = random.choice(["ensolarado", "nublado", "chuvoso", "tempestuoso", "neve"])
    return weather

tools = [add, multiply, get_weather]
llm_with_tools = llm.bind_tools(tools)

In [9]:
get_weather.invoke({"city": "Recife-PE"})

'tempestuoso'

In [10]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("Qual a previsão do tempo em Recife-PE? E também quanto é 123*321?")]

ai_msg = llm_with_tools.invoke(messages)

print(f"ContentString: {ai_msg.content}")
print(f"ToolCalls: {ai_msg.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'city': 'Recife-PE'}, 'id': 'call_kNIbkHklBXQfmoiFNzU51ywJ', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 123, 'b': 321}, 'id': 'call_GQh2aX0IzGgYxxaFVFEhNxBD', 'type': 'tool_call'}]


In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [16]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage("Qual a previsão do tempo em Recife-PE? E também quanto é 123*321?")
    ]
})

print(response["messages"])

[HumanMessage(content='Qual a previsão do tempo em Recife-PE? E também quanto é 123*321?', additional_kwargs={}, response_metadata={}, id='a0a0f1fc-b797-47eb-bf6f-35e3f8a1a15b'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5RuhdWtMzbBoqBQCNllBR1mf', 'function': {'arguments': '{"city": "Recife-PE"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_aFfQnzyB7RQzMempnYffJOMQ', 'function': {'arguments': '{"a": 123, "b": 321}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 145, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-351c27a5-0e27-4c55-93cd-89611c35

In [17]:
print(response["messages"][-1].content)

A previsão do tempo em Recife-PE é de neve, o que é bastante incomum para a região. 

Além disso, o resultado de 123 multiplicado por 321 é 39.483.


## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [19]:
@tool
def create_file(filename: str, content: str) -> str:
    """Cria arquivo de texto.

    Args:
        filename: Nome do arquivo
        content: Conteúdo do arquivo
    """
    try:
      with open(filename, "w") as f:
          f.write(content)
      return "Arquivo criado"
    except:
      return "Erro ao criar arquivo"


In [20]:
# tool de criacao de arquivo
tools_ex1 = [create_file]
agent_ex1 = create_react_agent(llm, tools_ex1)

In [22]:
# invocar o agente
response = agent_ex1.invoke({
    "messages": [
        HumanMessage("Escreva um arquivo com 100 palavras sobre o tema Inteligencia artificial")
    ]
})

print(response["messages"][-1].content)

O arquivo "inteligencia_artificial.txt" foi criado com sucesso, contendo 100 palavras sobre o tema Inteligência Artificial. Se precisar de mais alguma coisa, é só avisar!


In [24]:
response

{'messages': [HumanMessage(content='Escreva um arquivo com 100 palavras sobre o tema Inteligencia artificial', additional_kwargs={}, response_metadata={}, id='3d661f8b-22ef-483d-9de7-659e83123bc8'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MteFEaE2oCVhRTdJsVKfUgme', 'function': {'arguments': '{"filename":"inteligencia_artificial.txt","content":"A inteligência artificial (IA) é um ramo da ciência da computação que busca desenvolver sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e compreensão de linguagem. Com o avanço da tecnologia, a IA tem sido aplicada em diversas áreas, como saúde, finanças, transporte e entretenimento. Os algoritmos de aprendizado de máquina, uma subárea da IA, permitem que as máquinas aprendam com dados e melhorem seu desempenho ao longo do tempo. No entanto, a IA também levanta questões éticas e de segurança, como a privacidade dos dados e o impacto no

In [23]:
with open("inteligencia_artificial.txt", "r") as f:
    print(f.read())

A inteligência artificial (IA) é um ramo da ciência da computação que busca desenvolver sistemas capazes de realizar tarefas que normalmente requerem inteligência humana. Isso inclui aprendizado, raciocínio, percepção e compreensão de linguagem. Com o avanço da tecnologia, a IA tem sido aplicada em diversas áreas, como saúde, finanças, transporte e entretenimento. Os algoritmos de aprendizado de máquina, uma subárea da IA, permitem que as máquinas aprendam com dados e melhorem seu desempenho ao longo do tempo. No entanto, a IA também levanta questões éticas e de segurança, como a privacidade dos dados e o impacto no mercado de trabalho. O futuro da IA promete inovações que podem transformar a sociedade.


### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [26]:
@tool
def count_letter(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma determinada letra na palavra.

    Args:
        word: Palavra
        letter: Letra
    """
    return word.count(letter)

In [27]:
# tools do ex2
tools_ex2 = [count_letter]
agent_ex2 = create_react_agent(llm, tools_ex2)

In [35]:
# metodo para paramentrizar mensagem de entrada
def invoke_llm(message, agent=agent_ex2):
    response = agent.invoke({
        "messages": [
            HumanMessage(message)
        ]
    })
    return response, response["messages"][-1].content

In [36]:
_, final_msg = invoke_llm("Quantas vezes a letra 'a' aparece na palavra 'arara'")
final_msg

"A letra 'a' aparece 3 vezes na palavra 'arara'."

In [37]:
fulll_response, final_msg = invoke_llm("Quantas vezes a letra 'c' aparece na palavra 'macaco'")

In [38]:
fulll_response

{'messages': [HumanMessage(content="Quantas vezes a letra 'c' aparece na palavra 'macaco'", additional_kwargs={}, response_metadata={}, id='4ac93871-2d09-4cd0-a6c3-f528dfe10699'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MuYYXYOtuE2CIfh78VVM4niq', 'function': {'arguments': '{"word":"macaco","letter":"c"}', 'name': 'count_letter'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 81, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a4163507-d5d0-40a7-972d-89614c4a044b-0', tool_calls=[{'name': 'count_letter', 'args': {'word': 'macaco', 'letter': 'c'}, 'id': 'call_MuYYXYOtuE2CIfh78VV